In [143]:
# importing libraries
import os
import glob
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gensim.models import Word2Vec

import sklearn
import tensorflow as tf
import torch


# ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [144]:
task = 'task1'
train_path = './data/train/'
test_path = './data/test/sessions_test_' + task + '.csv'
PREDS_PER_SESSION = 100
slice_size = 1000

# slicing data for memory management
if slice_size != None:
    train_prod = pd.read_csv(train_path + '/products_train.csv', nrows=slice_size)
    train_sess = pd.read_csv(train_path + '/sessions_train.csv', nrows=slice_size)
    test_sess = pd.read_csv(test_path, nrows=slice_size)
else:
    train_prod = pd.read_csv(train_path + '/products_train.csv')
    train_sess = pd.read_csv(train_path + '/sessions_train.csv')
    test_sess = pd.read_csv(test_path)

In [145]:
train_prod.head(1)

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...


In [146]:
train_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        1000 non-null   object 
 1   locale    1000 non-null   object 
 2   title     1000 non-null   object 
 3   price     1000 non-null   float64
 4   brand     981 non-null    object 
 5   color     748 non-null    object 
 6   size      572 non-null    object 
 7   model     467 non-null    object 
 8   material  576 non-null    object 
 9   author    49 non-null     object 
 10  desc      931 non-null    object 
dtypes: float64(1), object(10)
memory usage: 86.1+ KB


In [147]:
train_sess.head(1)

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE


In [148]:
train_sess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prev_items  1000 non-null   object
 1   next_item   1000 non-null   object
 2   locale      1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [149]:
test_sess.head(1)

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE


In [150]:
test_sess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prev_items  1000 non-null   object
 1   locale      1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [151]:
import pandas as pd
import unicodedata

train_prod_txtcols = ['title', 'brand', 'color', 'size', 'model', 'material', 'author', 'desc']
train_prod_subcols = train_prod_txtcols

# Converting train_prod_subcols type to string
for col in train_prod_subcols:
    train_prod[col] = train_prod[col].astype(str)

special_chars = set()
for col in train_prod_subcols:
    for text in train_prod[col]:
        special_chars = special_chars.union(set(text))

# Writing special_chars to a text file, each character on a new line and comma separated
with open('special_chars.txt', 'w') as f:
    for i, item in enumerate(special_chars):
        if i == len(special_chars) - 1:
            f.write(item)
        else:
            f.write(item + ',')
    f.close()

# Read special characters from file
with open("special_chars.txt", "r") as f:
    special_chars = f.read().strip().split(',')

# Categorize special characters
categories = {}
for char in special_chars:
    if not char:
        continue
    category = unicodedata.category(char)
    if category not in categories:
        categories[category] = []
    categories[category].append(char)

# Output special characters by category to a json file
import json

with open("special_chars.json", "w") as f:
    json.dump(categories, f)

In [ ]:
# # showing row that has '$' or '#' or '@' or '%' or '|' in less_prod_text_cols
# count1 = train_prod_new[train_prod_new['title'].str.contains('\$|#|@|%|\|', regex=True)]

# # showing top 5 rows that have '$' in less_prod_text_cols
# count2 = train_prod_new[train_prod_new['title'].str.contains('\@', regex=True)].head()
# print(count2)

In [ ]:
# import re

# # Define a dictionary to map special characters to their textual value
# special_char_mapping = {'@': 'at', '$': 'USD', '%': 'percent'}

# # Iterate over each row and column in the DataFrame
# for idx, row in train_prod_new.iterrows():
#     for col in train_prod_new.columns:
#         # Get the cell value
#         cell_value = row[col]
#         if isinstance(cell_value, str):
#             # Check for special characters in the cell
#             special_chars = re.findall(r'[^\w\s]', cell_value)
#             for char in special_chars:
#                 # Check if the special character is in the mapping dictionary
#                 if char in special_char_mapping:
#                     # Replace the special character with its textual value
#                     cell_value = cell_value.replace(char, special_char_mapping[char])
#                 else:
#                     # Print the row, column, special character, and cell value for manual inspection
#                     print(f'Row {idx}, Column {col}: Special character {char} found with value {cell_value}')
#             # Update the cell value in the DataFrame
#             train_prod_new.loc[idx, col] = cell_value

# # showing top 5 rows that have '$' in less_prod_text_cols
# count2 = train_prod_new[train_prod_new['title'].str.contains('\@', regex=True)].head()

In [ ]:
# import re

# # Replace special characters
# train_prod_new['title'] = train_prod_new['title'].str.replace('@', 'at')
# train_prod_new['title'] = train_prod_new['title'].str.replace('$', 'USD')
# train_prod_new['title'] = train_prod_new['title'].str.replace('%', 'percent')

# # Identify other special characters
# special_chars = set()
# for title in train_prod_new['title']:
#     special_chars.update(set(re.findall('[^A-Za-z0-9\s]', title)))
# special_chars

# # printing special_chars to a text file
# with open('special_chars.txt', 'w') as f:
#     for item in special_chars:
#         f.write("%s " % item)

In [ ]:
# # data preprocessing
# train_prod_new['title'] = train_prod_new['title'].str.lower()
# train_prod_new['title'] = train_prod_new['title'].str.replace('[^a-z0-9 ]', '')
# train_prod_new['title'] = train_prod_new['title'].str.replace(' +', ' ')
# train_prod_new['title'] = train_prod_new['title'].str.strip()
# train_prod_new.head()

In [ ]:
# # showing row index=54188
# train_prod_new.iloc[1141]

In [ ]:
# textual_data = train_prod[['brand', 'color', 'size', 'model', 'material', 'author', 'desc']].values.flatten()
# textual_data = [text.lower().replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('-', ' ').replace('/', ' ').replace('"', ' ').replace("'", ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('{', ' ').replace('}', ' ').replace('<', ' ').replace('>', ' ').replace('\\', ' ').replace('@', ' ').replace('#', ' ').replace('$', ' ').replace('%', ' ').replace('^', ' ').replace('&', ' ').replace('*', ' ').replace('+', ' ').replace('=', ' ').replace('!', ' ').replace('?', ' ').replace('~', ' ').replace('`', ' ').split() for text in textual_data]

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# training a Word2Vec model on the textual columns
corpus = [train_prod_new[col].astype(str).values.tolist() for col in train_prod_new.columns]
corpus = [item for sublist in corpus for item in sublist]
corpus = [item.split() for item in corpus]

model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)

# # converting missing text values to vectors using Word2Vec
# for col in train_prod_new.columns:
#     col_vectors = []
#     for text in train_prod_new

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# Train a Word2Vec model on the textual columns
corpus = [train_prod[col].dropna().tolist() for col in ['brand', 'color', 'size', 'model', 'material', 'author', 'desc']]
model = Word2Vec(corpus, vector_size=100, window=5, min_count=5, workers=4, sg=1)

# Convert missing text values to vectors using Word2Vec
for col in ['brand', 'color', 'size', 'model', 'material', 'author', 'desc']:
    col_vectors = []
    for text in train_prod[col].fillna('').tolist():
        tokens = text.split()
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        if len(vectors) > 0:
            col_vectors.append(np.mean(vectors, axis=0))
        else:
            col_vectors.append(np.zeros(100)) # Use a zero vector if all tokens are out-of-vocabulary
    train_prod[col + '_vectors'] = col_vectors

# Impute missing values using clustering
from sklearn.cluster import KMeans

for col in ['brand', 'color', 'size', 'model', 'material', 'author', 'desc']:
    if train_prod[col].isna().sum() > 0:
        X = np.stack(train_prod[col + '_vectors'])
        kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
        cluster_assignments = kmeans.predict(X)
        missing_mask = train_prod[col].isna()
        missing_idx = np.where(missing_mask)[0]
        train_prod.loc[missing_mask, col] = np.array([kmeans.cluster_centers_[i] for i in cluster_assignments[missing_mask]]).reshape(missing_idx.shape[0], -1)

In [ ]:
# Load and preprocess the textual data
textual_data = train_prod[['brand', 'color', 'size', 'model', 'material', 'author', 'desc']].values.flatten()
textual_data = [text.lower().replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('-', ' ').replace('/', ' ').replace('"', ' ').replace("'", ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('{', ' ').replace('}', ' ').replace('<', ' ').replace('>', ' ').replace('\\', ' ').replace('@', ' ').replace('#', ' ').replace('$', ' ').replace('%', ' ').replace('^', ' ').replace('&', ' ').replace('*', ' ').replace('+', ' ').replace('=', ' ').replace('!', ' ').replace('?', ' ').replace('~', ' ').replace('`', ' ').split() for text in textual_data]

In [ ]:
# Train the Word2Vec model
model = Word2Vec(textual_data, size=100, min_count=1, window=5, workers=4)

# Represent missing values as vectors
for col in ['brand', 'color', 'size', 'model', 'material', 'author', 'desc']:
    col_vectors = []
    for text in train_prod[col]:
        if pd.isna(text):
            col_vectors.append(np.zeros(100)) # Use a zero vector for missing values
        else:
            tokens = text.lower().replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('-', ' ').replace('/', ' ').replace('"', ' ').replace("'", ' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('{', ' ').replace('}', ' ').replace('<', ' ').replace('>', ' ').replace('\\', ' ').replace('@', ' ').replace('#', ' ').replace('$', ' ').replace('%', ' ').replace('^', ' ').replace('&', ' ').replace('*', ' ').replace('+', ' ').replace('=', ' ').replace('!', ' ').replace('?', ' ').replace('~', ' ').replace('`', ' ').split()
            col_vectors.append(np.mean([model.wv[token] for token in tokens], axis=0))
    train_prod[col] = col_vectors


In [ ]:
train_prod_nulls.head(5)

In [ ]:
for col in train_prod_nulls.columns:
    if col == 'id':
        continue
    else:
        uniques = train_prod_nulls[col].unique()
        for item in uniques:
            print(col, item, train_prod_nulls[col].value_counts()[item])

In [ ]:
def null_saver(df, path):
    null = df[df.isnull().any(axis=1)]
    if len(null) > 0:
        print(f'Found {len(null)} null rows in {df.shape[0]} rows')
        print(null.head())
        # null.to_csv(path, index=False)
        print(f'Saved {len(null)} null rows to {path}')
    else:
        print(f'No null rows found in {df}')

null_saver(train_prod, './data/train/products_train-null.csv')
null_saver(train_sess, './data/train/sessions_train-null.csv')
null_saver(test_sess, './data/test/sessions_test_task1-null.csv')